In [83]:
import pandas as pd
import numpy as np
from fredapi import Fred
import yfinance as yf
import glob

In [84]:
start_date = '2020-09-01'
end_date = '2024-09-01'

In [85]:
# NVDA stock price
target = ['NVDA']
stock_price= yf.download(target, start_date, end_date)['Adj Close']
stock_price.index = stock_price.index.strftime('%Y%m%d')
stock_price.to_csv('./Data/nvidia_stock_price.csv')

[*********************100%***********************]  1 of 1 completed

In [86]:
# Competitors stock price
competitors = ['AMD', 'INTC', 'QCOM']
stock_price_competitors = yf.download(competitors, start_date, end_date)['Adj Close']
stock_price_competitors.index = stock_price_competitors.index.strftime('%Y%m%d')
stock_price_competitors.to_csv('./Data/competitors_stock_price.csv')


[*********************100%***********************]  3 of 3 completed


In [87]:
# ADS INDEX
data_ads = pd.read_excel('ADS_Index_Most_Current_Vintage.xlsx')
data_ads['date'] = pd.to_datetime(data_ads['date'])
data_ads = data_ads[(data_ads['date'] >= start_date) & (data_ads['date'] <= end_date)]
data_ads['date'] = data_ads['date'].dt.strftime('%Y%m%d')
df_ads = data_ads.set_index('date')
df_ads.to_csv('./Data/ADS_Index_Processed.csv')


In [88]:
# Fama French 5 Factors
data_ff5 = pd.read_csv('FF_Research_Data_5_Factors_daily.csv')
data_ff5['date'] = data_ff5['date'].astype(str).str[0:4] + '-' + \
                   data_ff5['date'].astype(str).str[4:6] + '-' + \
                   data_ff5['date'].astype(str).str[6:8]
data_ff5['date'] = pd.to_datetime(data_ff5['date'])
filtered_ff5 = data_ff5[(data_ff5['date'] >= start_date) & (data_ff5['date'] <= end_date)]
filtered_ff5 = filtered_ff5[filtered_ff5['date'].dt.weekday < 5]
filtered_ff5['date'] = filtered_ff5['date'].dt.strftime('%Y%m%d')
filtered_ff5 = filtered_ff5.set_index('date')
filtered_ff5.to_csv('./Data/FF5_Factors_Processed.csv')


In [89]:
# Initialize Fred API
fred = Fred(api_key='96b8dcff4c5efa52d89e588594f2930c')

# Define the list of series
varList = [
    'T10Y3M', 'DGS10', 'OBMMIJUMBO30YF',  # Term premium 10yr-3mon, 30yr mortgage jumbo loan
    'DEXUSEU', 'DEXJPUS', 'DEXUSUK',      # Spot exchange rates to EUR, JPY, GBP
    'CBBTCUSD', 'CBETHUSD',               # Cryptocurrencies
    'T10YIE', 'DCOILBRENTEU',             # Breakeven inflation, Brent oil price
    'VIXCLS',                             # Implied volatilities
    'DAAA', 'DBAA',                       # Corporate bond yield
    'AMERIBOR', 'T5YIE', 'BAMLH0A0HYM2',  # Additional financial indicators
    'BAMLH0A0HYM2EY', 'DGS1', 'DCOILWTICO',
    'DHHNGSP'
]

# Fetch SP500 series and initialize DataFrame
SP500 = fred.get_series('SP500')
SP500.name = 'SP500'
df_fred = SP500.to_frame()

# Merge all series into one DataFrame
for var in varList:
    try:
        data = fred.get_series(var)
        data.name = var
        df_fred = pd.merge(df_fred, data.to_frame(), left_index=True, right_index=True, how='outer')
    except Exception as e:
        print(f"Failed to fetch {var}: {e}")

# Format date index as YYYYMMDD
df_fred.index = df_fred.index.strftime('%Y%m%d')
df_fred = df_fred[(df_fred.index >= start_date) & (df_fred.index <= end_date)]
df_fred.to_csv('./Data/fred_data_processed.csv', index_label='date')

print("Fred data successfully saved to './Data/fred_data_processed.csv'")


Fred data successfully saved to './Data/fred_data_processed.csv'


In [90]:
SP500 = fred.get_series('SP500', start_date=start_date, end_date=end_date)
stock_open_dates = SP500.index.strftime('%Y%m%d')  
stock_open_dates = pd.Series(stock_open_dates)

btc = yf.download('BTC-USD', start=start_date, end=end_date)['Adj Close']
btc = btc.reset_index()  # Convert to DataFrame
btc['Date'] = btc['Date'].dt.strftime('%Y%m%d')  # Format dates as YYYYMMDD
btc.set_index('Date', inplace=True)

# Filter BTC prices for stock market open dates
btc_filtered = btc[btc.index.isin(stock_open_dates)]

# Save filtered BTC prices to CSV
btc_filtered.to_csv('./Data/BTC_Stock_Market_Open.csv', index_label='date')

print("Filtered BTC prices saved to './Data/BTC_Stock_Market_Open.csv'")

[*********************100%***********************]  1 of 1 completed

Filtered BTC prices saved to './Data/BTC_Stock_Market_Open.csv'


In [94]:
import pandas as pd
from fredapi import Fred

# Initialize Fred API
fred = Fred(api_key='96b8dcff4c5efa52d89e588594f2930c')

# Define time range
start_date = '2023-09-01'
end_date = '2024-09-01'

# Step 1: Fetch stock market open dates
SP500 = fred.get_series('SP500', start_date=start_date, end_date=end_date)
stock_open_dates = SP500.index.strftime('%Y%m%d')  # Format as YYYYMMDD
stock_open_dates = set(stock_open_dates)  # Convert to a set for fast filtering

# Step 2: Define your CSV files
csv_files = [
    './Data/ADS_Index_Processed.csv',
    './Data/nvidia_stock_price.csv',
    './Data/competitors_stock_price.csv',
    './Data/FF5_Factors_Processed.csv',
    './Data/fred_data_processed.csv',
    './Data/BTC_Stock_Market_Open.csv'
]

# Step 3: Read, filter, and merge the data
merged_data = None

for file in csv_files:
    # Read CSV file
    df = pd.read_csv(file)
    
    # Ensure 'date' column exists and is in string format
    if 'date' in df.columns:
        df.rename(columns={'date': 'date'}, inplace=True)
    elif 'Date' in df.columns:
        df.rename(columns={'Date': 'date'}, inplace=True)
    else:
        raise ValueError(f"Neither 'Date' nor 'date' column exists in {file}")
    
    df['date'] = df['date'].astype(str)
    
    # Filter for stock market open dates
    df = df[df['date'].isin(stock_open_dates)]
    
    # Merge the data
    if merged_data is None:
        merged_data = df
    else:
        merged_data = pd.merge(merged_data, df, on='date', how='outer')

# Step 4: Additional filter after merging
merged_data = merged_data[(merged_data['date'] >= start_date.replace("-", "")) & 
                          (merged_data['date'] <= end_date.replace("-", ""))]

# Step 5: Save the merged data to a new CSV file
merged_data.to_csv('./Features.csv', index=False)

print("Merged data saved to './Features.csv'")


Merged data saved to './Features.csv'
